In [ ]:
import http.server
import socketserver
import networkx as nx
import pandas as pd
import json
import base64
import math

df = pd.read_json('hospitales_data_final.json')
cdf = pd.read_json('centroides_final.json')

def calcular_distancia_euclidean(cord1,cord2):
    return math.sqrt((cord1[0]-cord2[0])**2 +(cord1[1]-cord2[1])**2+(cord1[2]-cord2[2])**2) #Calcular la distancia entre dos puntos con coordenadas cartesianas x,y,z


PORT = 8000

class MyRequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.end_headers()
            fh=open('index.html','rb')
            string=fh.read()
            self.wfile.write(string)
        elif self.path.find('hospitales_data_final')>0:
            self.send_response(200)
            self.send_header('Content-Type', 'application/json')
            self.end_headers()
            fh=open('hospitales_data_final.json','rb')
            string=fh.read()
            self.wfile.write(string) 
        elif self.path.find('hospital42.png')>0:
            self.send_response(200)
            self.send_header('Content-Type', 'image/png')
            self.end_headers()
            fh=open('hospital42.png','rb')
            string=fh.read()
            self.wfile.write(string)     
        elif self.path.find('centroides_final')>0:
            self.send_response(200)
            self.send_header('Content-Type', 'application/json')
            self.end_headers()
            fh=open('centroides_final.json','rb')
            string=fh.read()
            self.wfile.write(string)               
        elif self.path.find('request_min_path')>0:
            self.send_response(200)
            self.send_header('Content-Type', 'application/json')
            self.end_headers()
            values = {}
            
            if '?' in self.path:
                data = self.path.split('?', 1)[1].split('&')
                for n in data:
                    d = n.split("=")
                    values[d[0]] = float(d[1])    
              
            data = {}
            maxima_distancia = values["distancia_maxima"]
            if len(values.keys())>0 :
                if values["data"]==0:
                    G=nx.Graph()
                    for index, row in df.iterrows():
                        G.add_node(row["id"],pos=(row["cartesianas"][0],row["cartesianas"][1]))
                        if row["distancia_centro"] !=0 :
                            G.add_edge(row["hospitales_centros"],row["id"],weight=row["distancia_centro"])

                    for index, row in cdf.iterrows():
                        if row["distancias"] < maxima_distancia:
                            G.add_edge(row["inicio"],row["final"],weight=row["distancias"])

                    rute=[]
                    try:
                        rute=nx.shortest_path(G, values["inicio"],values["final"],weight="weight")
                    except nx.NetworkXNoPath:
                        print("No path")
                    data = {"ruta":rute}

                else:
                    G2=nx.Graph()
                    print("iniciando ...")
                    for index, row in df.iterrows():
                        G2.add_node(row["id"],pos=(row["cartesianas"][0],row["cartesianas"][1]))
                    cantidad_de_nodos=len(df["cartesianas"])
                    print(calcular_distancia_euclidean(df["cartesianas"][values["inicio"]],df["cartesianas"][values["final"]]))
                    for i in range(0,cantidad_de_nodos):
                        for j in range(0,cantidad_de_nodos):
                            if calcular_distancia_euclidean(df["cartesianas"][i],df["cartesianas"][j])<maxima_distancia and i!=j:
                                G2.add_edge(i,j,weight=calcular_distancia_euclidean(df["cartesianas"][i],df["cartesianas"][j]))
                    print("terminando ...")
                    rute=[]
                    try:
                        rute=nx.shortest_path(G2, values["inicio"],values["final"],weight="weight")
                    except nx.NetworkXNoPath:
                        print("No path")
                    data = {"ruta":rute}
            json_object = json.dumps(data)  
            self.wfile.write(bytes(json_object, "utf8"))
        return 

Handler = MyRequestHandler
with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print("serving at port", PORT)
    httpd.serve_forever()


serving at port 8000


127.0.0.1 - - [13/Jul/2021 17:07:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:07:53] "GET /hospitales_data_final HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:07:53] "GET /hospital42.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:07:55] "GET /request_min_path?inicio=0&final=7&data=0&distancia_maxima=12 HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:11] "GET /hospitales_data_final HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:11] "GET /hospital42.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:13] "GET /request_min_path?inicio=0&final=7&data=0&distancia_maxima=12 HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:56] "GET /hospitales_data_final HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:56] "GET /hospital42.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:08:57] "GET /request_min_path?inicio=0&final=8&data=0&distancia_maxima=12 HTTP/1.1" 2

No path


127.0.0.1 - - [13/Jul/2021 17:12:26] "GET /request_min_path?inicio=0&final=15&data=0&distancia_maxima=14 HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:14:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:14:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:14:33] "GET /hospitales_data_final HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:14:33] "GET /hospital42.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:14:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:15:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:15:25] "GET /hospitales_data_final HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:15:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:16:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:16:48] "GET /hospitales_data_final HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:16:49] "GET /hospital42.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2021 17:16:52] "GET /request_min_path?inicio=0&final=8&data=0&distancia_maxima=12 HTTP/1.1" 200 -
127.0.0.